In [1]:
import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import time
import warnings

from premade_funcs import load_data

from sklearn.neural_network import MLPClassifier
import sklearn.metrics

Just load_and_plot_data code here:

In [2]:
data_dir = os.path.abspath("data_fashion/")
test_x, train_x, train_y_df, valid_x, valid_y_df = load_data(data_dir)

Contents of train_x.csv: arr of shape (2102, 784)
Contents of valid_x.csv: arr of shape (600, 784)
Contents of test_x.csv: arr of shape (600, 784)


## Pre-set values to put into MLPClassifier

In [9]:
n_runs = 4
max_iter_per_run = 1000
tr_balacc_arr = 0.11 * np.ones((1, n_runs)) # Pre allocate with arbitrary values, just so code doesnt break
te_balacc_arr = 0.22 * np.ones((1, n_runs))

NameError: name 'S' is not defined

In [ ]:
for run_id, random_state in enumerate(range(n_runs)):
    
    start_time_sec = time.time()
    mlp = MLPClassifier(
        hidden_layer_sizes=1,   #At MOST 1 hidden layer per spec.
        activation='relu',
        alpha=0.0001,
        tol=1e-5,
        max_iter=max_iter_per_run,
        random_state=random_state,
        solver='lbfgs', batch_size=10000,
        )
    
    with warnings.catch_warnings(record=True) as warn_list:
        mlp.fit(train_x, train_y_df)
    elapsed_time_sec = time.time() - start_time_sec
    mlp.did_converge = True if len(warn_list) == 0 else False

    # TODO compute the error rate on the training set and test set
    # Can use code like this:
    #      sklearn.metrics.zero_one_loss(..., ...)
    tr_acc = 
    te_acc = sklearn.metrics.zero_one_loss(y_te_T, mlp.predict(x_te_T2))
    
    tr_err_arr[size_list.index(size), run_id] = tr_err
    te_err_arr[size_list.index(size), run_id] = te_err

    # TODO store this mlp's performance so you can plot it later
    # You can use the provided `tr_loss_arr', or find another way that makes sense to you
    tr_loss_arr[size_list.index(size), run_id] = tr_log_loss
    te_loss_arr[size_list.index(size), run_id] = te_log_loss
    
    print(
        'run %d/%d done after %5.1f sec | %13s after %3d iter | tr err %.3f | te err %.3f' % (
        size, 
        run_id + 1, n_runs, elapsed_time_sec,
        'yes converged' if mlp.did_converge else 'NOT converged',
        max_iter_per_run,
        tr_err, te_err))